In [2]:
#各ライブラリのimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#XGBoost
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)
#pwrカラム作成
#df_train['pwr'] = df_train['weight']/df_train['horsepower']
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正
#エンコーディング
df_number = pd.get_dummies(df_train , columns = ['origin' , 'manufacturers name'] , dtype=int)
#特徴量の分割
features = df_number.drop(['id' , 'cylinders' ,'weight' , 'car name' , 'mpg'] , axis = 1)
#print(features)
target = df_train['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor(n_estimators = 100)
model.fit(features_train , target_train)
pred = model.predict(features_test)
#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 
#pwrカラムなし,horsepowerあり,accelerationあり;2.92
#pwrカラムなし,horsepowerなしaccelerationあり;3.098
#pwrカラムあり,horsepowerなし,accelerationあり;2.996
#pwrカラムあり,horsepowerなし,accelerationなし;3.390

3.0805420080889037

In [3]:
#XGBoost
#gridsearch
#読み込みから標準化まで(trainのみ)
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)
#pwrカラム作成
df_train['pwr'] = df_train['weight']/df_train['horsepower']
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正
#エンコーディング
df_number = pd.get_dummies(df_train , columns = ['origin' , 'manufacturers name'] , dtype=int)
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_number.drop(['mpg','id' , 'car name' , 'weight' , 'cylinders' , 'horsepower'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)


#特徴量の分割
features = df_train_std
target = df_train['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor()
#gridsearch
from sklearn.model_selection import GridSearchCV

# 最終的なパラメータ範囲
cv_params = {'n_estimators': [20,40,60,80,100],
             'learning_rate': [0.01, 0.03, 0.1, 0.3],#学習率(0～1)
             'min_child_weight': [2, 4, 6, 8],#決定木の葉の重みの下限
             'max_depth': [1, 2, 3, 4],#決定木の最大深度(整数)
             'colsample_bytree': [0.2, 0.5, 0.8, 1.0],#説明変数のサンプル抽出比(木)(0~1)
             'subsample': [0.2, 0.5, 0.8, 1.0]#各決定木のサンプル抽出比(0~1)、小さいほど保守的になる
             }
# グリッドサーチのインスタンス作成
#引数一覧
#estimator: チューニングを行うモデル
#param_grid: パラメータ候補パラメータ名: [候補リスト]
#scoring: 評価指標(今回はneg_mean_squared_error(RSME))
#cv: Cross Validationの分割数(default: 3)
#verbose: ログ出力レベル
#n_jobs: 同時実行数(-1: コア数で並列実行)
#refit: 　trueのとき最良のパラメータで再学習
grid_model = GridSearchCV(model, cv_params,cv = 5,
                      scoring='neg_mean_squared_error', n_jobs=-1)
grid_model.fit(features_train , target_train)
pred = grid_model.predict(features_test)
#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 
#pwrカラムなし,horsepowerあり,accelerationあり;2.62
#pwrカラムなし,horsepowerなしaccelerationあり;2.72
#pwrカラムあり,horsepowerなし,accelerationあり;2.53
#pwrカラムあり,horsepowerなし,accelerationなし;2.85
#pwrカラムあり,horsepowerあり,accelerationなし;3.26

2.6592625087884603

In [13]:
#モデル実装(train,test結合版)

#gridsearchなし
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成
df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
df_number = pd.get_dummies(df_train_test , columns = ['origin' , 'manufacturers name'] , dtype = int)
#print(df_number)
df_train = df_number.iloc[:198 , :]
df_test = df_number.iloc[199: , :]

#目的変数の分離
features_train = df_train.drop(['weight', 'id' , 'car name' , 'mpg','horsepower' , 'cylinders','acceleration'] , axis = 1)
target_train = df_train['mpg']

features_test = df_test.drop(['weight' , 'id' , 'car name' , 'mpg','horsepower' , 'cylinders','acceleration'] , axis = 1)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor()
model.fit(features_train , target_train)

df_pred = pd.DataFrame(model.predict(features_test))
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('XGBsample_submit.csv' , header = False , index = False)

In [6]:
#モデル実装(train,test結合版)#実装途中


#gridsearch版
#kfoldを入れ、pwrカラムを追加しない時が一番高かった。
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成in_test['displacement']
#df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
df_number = pd.get_dummies(df_train_test , columns = ['origin' , 'manufacturers name'] , dtype = int)
df_train = df_number.iloc[:198 , :]
df_test = df_number.iloc[199: , :]
#学習
#import
import xgboost as xgb
from xgboost import XGBRegressor
#データ変換
features_train = xgb.DMatrix(df_train.drop([ 'id' , 'car name' , 'mpg'] , axis = 1) , )
#目的変数の分離
features_train = 
target_train = df_train['mpg']
print(target_train.head())
features_test = df_test.drop([  'id' , 'car name' , 'mpg'] , axis = 1)
#評価用データ
from sklearn.model_selection import train_test_split
train_train , train_eval , target_target , target_eval  = train_test_split(features_train , target_train , test_size=0.2 , random_state = 0)

#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor(booster='gbtree', objective='reg:squarederror',
                     random_state=42, n_estimators=10000 )
fit_params = {'verbose': 0,  # 学習中のコマンドライン出力
              'early_stopping_rounds': 10,  # 学習時、評価指標がこの回数連続で改善しなくなった時点でストップ
              'eval_metric': 'rmse',  # early_stopping_roundsの評価指標
              'eval_set': [(train_train, target_eval)]  # early_stopping_roundsの評価指標算出用データ
              }
# 学習時fitパラメータ指定
#fit_params = {'verbose': 0,  # 学習中のコマンドライン出力
              #'early_stopping_rounds': 10,  # 学習時、評価指標がこの回数連続で改善しなくなった時点でストップ
              #'eval_metric': 'rmse',  # early_stopping_roundsの評価指標
              #}
#K-分割交差検証

#交差検証は比較的少ないデータセットを使って学習する場合に、過学習を防ぐ（汎化性能を上げる）ためにする
from sklearn.model_selection import KFold
cv = KFold(n_splits=3, shuffle=True, random_state=42)
#gridsearch
from sklearn.model_selection import GridSearchCV

# 最終的なパラメータ範囲
cv_params = {'learning_rate': [0.01, 0.03, 0.1, 0.3],#学習率(0～1)
             'min_child_weight': [2, 4, 6, 8],#決定木の葉の重みの下限
             'max_depth': [1, 2, 3, 4],#決定木の最大深度(整数)
             'colsample_bytree': [0.2, 0.5, 0.8, 1.0],#説明変数のサンプル抽出比(木)(0~1)
             'subsample': [0.2, 0.5, 0.8, 1.0]#各決定木のサンプル抽出比(0~1)、小さいほど保守的になる
             }
# グリッドサーチのインスタンス作成
#引数一覧
#estimator: チューニングを行うモデル
#param_grid: パラメータ候補パラメータ名: [候補リスト]
#scoring: 評価指標(今回はneg_mean_squared_error(RSME))
#cv: Cross Validationの分割数(default: 3)
#verbose: ログ出力レベル
#n_jobs: 同時実行数(-1: コア数で並列実行)
#refit: 　trueのとき最良のパラメータで再学習

 
grid_model = GridSearchCV(model, cv_params,cv = cv,
                      scoring='neg_mean_squared_error', n_jobs=-1)
grid_model.fit(features_train , target_train )

df_pred = pd.DataFrame(grid_model.predict(features_test))
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('XGBsample_submit.csv' , header = False , index = False)

0    29.0
1    31.9
2    19.0
3    28.0
4    37.7
Name: mpg, dtype: float64


KeyboardInterrupt: 

In [5]:
#メーカーと国を合わせたカラム作成
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成
#df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正
#国とメーカーカラムの結合
df_train_test['manufacturers_country'] = df_train_test['manufacturers name'] + df_train_test['origin'].astype(str)
#エンコーディング
df_number = pd.get_dummies(df_train_test , columns = ['manufacturers_country'] , dtype = int)
df_train = df_number.iloc[:198 , :]
df_test = df_number.iloc[199: , :]

#目的変数の分離
features_train = df_train.drop([ 'id' , 'car name' , 'mpg' , 'cylinders','weight' , 'manufacturers name'] , axis = 1)
target_train = df_train['mpg']
print(target_train.head())
features_test = df_test.drop([  'id' , 'car name' , 'mpg', 'cylinders' , 'weight' ,'manufacturers name'] , axis = 1)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor()
#K-分割交差検証
#交差検証は比較的少ないデータセットを使って学習する場合に、過学習を防ぐ（汎化性能を上げる）ためにする
from sklearn.model_selection import KFold
cv = KFold(n_splits=3, shuffle=True, random_state=42)
#gridsearch
from sklearn.model_selection import GridSearchCV

# 最終的なパラメータ範囲
cv_params = {'n_estimators': [100 ,200 ,300],
             'learning_rate': [0.01, 0.03, 0.1, 0.3],#学習率(0～1)
             'min_child_weight': [2, 4, 6, 8],#決定木の葉の重みの下限
             'max_depth': [1, 2, 3, 4],#決定木の最大深度(整数)
             'colsample_bytree': [0.2, 0.5, 0.8, 1.0],#説明変数のサンプル抽出比(木)(0~1)
             'subsample': [0.2, 0.5, 0.8, 1.0]#各決定木のサンプル抽出比(0~1)、小さいほど保守的になる
             }
# グリッドサーチのインスタンス作成
#引数一覧
#estimator: チューニングを行うモデル
#param_grid: パラメータ候補パラメータ名: [候補リスト]
#scoring: 評価指標(今回はneg_mean_squared_error(RSME))
#cv: Cross Validationの分割数(default: 3)
#verbose: ログ出力レベル
#n_jobs: 同時実行数(-1: コア数で並列実行)
#refit: 　trueのとき最良のパラメータで再学習

 
grid_model = GridSearchCV(model, cv_params,cv = cv,
                      scoring='neg_mean_squared_error', n_jobs=-1)
grid_model.fit(features_train , target_train)
best = grid_model.best_estimator_
print(best)
df_pred = pd.DataFrame(grid_model.predict(features_test))
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('XGB_pwr_sample_submit.csv' , header = False , index = False)




0    29.0
1    31.9
2    19.0
3    28.0
4    37.7
Name: mpg, dtype: float64
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=4, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [30]:
#モデル実装(train,test結合版)#メーカーと国は別
#gridsearch版

#kfoldを入れ、pwrカラムを追加しない時が一番高かった。
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成
#df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
df_number = pd.get_dummies(df_train_test , columns = ['origin' , 'manufacturers name'] , dtype = int)
df_train = df_number.iloc[:198 , :]
df_test = df_number.iloc[199: , :]

#目的変数の分離
features_train = df_train.drop([ 'id' , 'car name' , 'mpg' , 'cylinders','weight' ] , axis = 1)
target_train = df_train['mpg']
print(target_train.head())
features_test = df_test.drop([  'id' , 'car name' , 'mpg', 'cylinders' , 'weight'] , axis = 1)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor()
#K-分割交差検証
#交差検証は比較的少ないデータセットを使って学習する場合に、過学習を防ぐ（汎化性能を上げる）ためにする
from sklearn.model_selection import KFold
cv = KFold(n_splits=3, shuffle=True, random_state=42)
#gridsearch
from sklearn.model_selection import GridSearchCV

# 最終的なパラメータ範囲
cv_params = {'n_estimators': [200,300,400],
             'learning_rate': [0.01, 0.03, 0.1, 0.3],#学習率(0～1)
             'min_child_weight': [2, 4, 6, 8],#決定木の葉の重みの下限
             'max_depth': [1, 2, 3, 4],#決定木の最大深度(整数)
             'colsample_bytree': [0.2, 0.5, 0.8, 1.0],#説明変数のサンプル抽出比(木)(0~1)
             'subsample': [0.2, 0.5, 0.8, 1.0]#各決定木のサンプル抽出比(0~1)、小さいほど保守的になる
             }
# グリッドサーチのインスタンス作成
#引数一覧
#estimator: チューニングを行うモデル
#param_grid: パラメータ候補パラメータ名: [候補リスト]
#scoring: 評価指標(今回はneg_mean_squared_error(RSME))
#cv: Cross Validationの分割数(default: 3)
#verbose: ログ出力レベル
#n_jobs: 同時実行数(-1: コア数で並列実行)
#refit: 　trueのとき最良のパラメータで再学習

 
grid_model = GridSearchCV(model, cv_params,cv = cv,
                      scoring='neg_mean_squared_error', n_jobs=-1)
grid_model.fit(features_train , target_train)
best = grid_model.best_estimator_
print(best)
df_pred = pd.DataFrame(grid_model.predict(features_test))
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('XGBsample_submit.csv' , header = False , index = False)
#29.31(300)pwrあり
#29.37



0    29.0
1    31.9
2    19.0
3    28.0
4    37.7
Name: mpg, dtype: float64
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=4, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [ ]:
#目的変数の分離
features_train = df_train.drop(['weight' , 'acceleration' , 'mpg' ] , axis = 1)
target_train = df_train['mpg']
features_test = df_test.drop(['weight' , 'acceleration'] , axis = 1)
#学習
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor(n_estimators = 100)
model.fit(features_train , target_train)
#予測
df_pred = pd.DataFrame(model.predict(features_test))
df_test = pd.read_csv('../test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('XGBsample_submit.csv' , header = False , index = False)

In [ ]:
#gridsearch
from sklearn.model_selection import GridSearchCV

model = GridSearchCV()
# 最終的なパラメータ範囲
cv_params = {'n_estimators': [20,40,60,80,100],
             'learning_rate': [0.01, 0.03, 0.1, 0.3],#学習率(0～1)
             'min_child_weight': [2, 4, 6, 8],#決定木の葉の重みの下限
             'max_depth': [1, 2, 3, 4],#決定木の最大深度(整数)
             'colsample_bytree': [0.2, 0.5, 0.8, 1.0],#説明変数のサンプル抽出比(木)(0~1)
             'subsample': [0.2, 0.5, 0.8, 1.0]#各決定木のサンプル抽出比(0~1)、小さいほど保守的になる
             }
# グリッドサーチのインスタンス作成
#引数一覧
#estimator: チューニングを行うモデル
#param_grid: パラメータ候補パラメータ名: [候補リスト]
#scoring: 評価指標(今回はneg_mean_squared_error(RSME))
#cv: Cross Validationの分割数(default: 3)
#verbose: ログ出力レベル
#n_jobs: 同時実行数(-1: コア数で並列実行)
#refit: 　trueのとき最良のパラメータで再学習
grid_model = GridSearchCV(model, cv_params,cv = 5,
                      scoring='neg_mean_squared_error', n_jobs=-1)
grid_model.fit(features_train , target_train)
